In [11]:
import utils
import models.categorical
import math
import pickle
import numpy as np
import pandas as pd
from datetime import date
import os.path
from os import path

from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [14]:
futuresList = utils.futuresList
y_var = "LONG_SHORT"
counter = 1
for future in tqdm(futuresList):
    # if path.exists(f"model_metrics/categorical/{file_dir}{future}.csv"): continue
    # print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = utils.generate_X_vars(future, linearise=False, \
                                   tech_indicators=[], \
                                   macro_indicators=True)
    for y in ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']:
        X_vars.append(y)
    
    print(future)
    print(X_vars)
    counter += 1
    if counter == 20: break
    # load X and y
    # X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    # X_df.to_csv(f"/Users/aeronsim/{future}.csv")

  1%|          | 1/88 [00:01<01:53,  1.31s/it]F_AD
['CLOSE_PCT', 'VOL_PCT', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
  2%|▏         | 2/88 [00:02<01:41,  1.18s/it]F_BO
['CLOSE_PCT', 'VOL_PCT', 'USA_CFNAI', 'USA_NFP', 'USA_CCPI', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
  3%|▎         | 3/88 [00:02<01:30,  1.07s/it]F_BP
['CLOSE_PCT', 'VOL_PCT', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
  5%|▍         | 4/88 [00:03<01:22,  1.02it/s]F_C
['CLOSE_PCT', 'VOL_PCT', 'USA_CFNAI', 'USA_NFP', 'USA_CCPI', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
  6%|▌         | 5/88 [00:04<01:17,  1.07it/s]F_CC
['CLOSE_PCT', 'VOL_PCT', 'USA_CFNAI', 'USA_NFP', 'USA_CCPI', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
  7%|▋         | 6/88 [00:05<01:12,  1.13it/s]F_CD
['CLOSE_PCT', 'VOL_PCT', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
  8%|▊         | 7/88 [00:06<01:09,  1.17it/s]F_CL
['CLOSE_PCT', 'VOL_PCT', 'USA_CFNAI', 'USA_NFP', 'USA_CCPI', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
  9%|▉         | 8/88 [00:06<01:06,  1.20it/s]F_CT
[

In [ ]:
fourCandleHammerParams = [
    {
        "N": [30, 50],
        "highFactor": [0.95, 0.85],
        "lowFactor": [1.05, 1.15]
    }
]

emaParams = [
    {
        "shortTermDays": [10, 20],
        "longTermDays": [30, 40],
        "NDays": [50, 60]
    }
]

swingSetupParams = [
    {
        "shortTermDays": [20, 30],
        "longTermDays": [40, 60],
        "NDays": [80, 100]
    }
]

start_date = date(2010, 1, 1)
end_date = date(2021, 1, 1)
futuresList = utils.futuresList

## Hyperparameter Tuning for FourCandleHammer Strategy

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(fourCandleHammerParams))
y_var = "LONG_SHORT"
file_dir = "fourCandleHammer/perc/"

for future in tqdm(futuresList):
    if path.exists(f"model_metrics/categorical/{file_dir}{future}.csv"): continue
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        df_with_strategy = utils.fourCandleHammer(X_df, param_set['N'], param_set['highFactor'], param_set['lowFactor'])
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            df_strategy = df_with_strategy, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]
        
    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

## Save FourCandleHammer Model

In [ ]:
for future in futuresList:
    # prepare X variables
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # save model
    # metric="opp_cost_SMA"
    models.categorical.save_model_techIndicators(path='fourCandleHammer/perc', metric="accuracy_SMA", 
                                                model_wrapper=models.categorical.fourCandleHammerWrapper, 
                                                future=future, X_vars=X_vars, y_var="LONG_SHORT")


## Hyperparameter Tuning for EMA Strategy

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(emaParams))
y_var = "LONG_SHORT"
file_dir = "ema/perc/"

for future in tqdm(futuresList):
    if path.exists(f"model_metrics/categorical/{file_dir}{future}.csv"): continue
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        if param_set['shortTermDays'] > param_set['longTermDays']: continue
        df_with_strategy = utils.ema_strategy(X_df, param_set['shortTermDays'], param_set['longTermDays'], param_set['NDays'])
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            df_strategy = df_with_strategy, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

## Save EMA Model

In [ ]:
for future in futuresList:
    # prepare X variables
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # save model
    # metric="opp_cost_SMA"
    models.categorical.save_model_techIndicators(path='ema/perc', metric="accuracy_SMA", 
                                                model_wrapper=models.categorical.emaStrategyWrapper, 
                                                future=future, X_vars=X_vars, y_var="LONG_SHORT")

## Hyperparameter Tuning for Swing Setup Strategy

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(swingSetupParams))
y_var = "LONG_SHORT"
file_dir = "swingSetup/perc/"

for future in tqdm(futuresList):
    if path.exists(f"model_metrics/categorical/{file_dir}{future}.csv"): continue
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        if param_set['shortTermDays'] > param_set['longTermDays']: continue
        df_with_strategy = utils.swing_setup(X_df, param_set['shortTermDays'], param_set['longTermDays'], param_set['NDays'])
        # print("Gotten strategy, now walk fwd")
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            df_strategy = df_with_strategy, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        # print("Done walk fwd")
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

## Save Swing Setup Model

In [ ]:
for future in futuresList:
    # prepare X variables
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # save model
    # metric="opp_cost_SMA"
    models.categorical.save_model_techIndicators(path='swingSetup/perc', metric="accuracy_SMA", 
                                                model_wrapper=models.categorical.swingSetupWrapper, 
                                                future=future, X_vars=X_vars, y_var="LONG_SHORT")